# __Scenario D:__<br>Time-dependent enzyme inactivation

Data provided by Paulo Durão (Microbial & Enzyme Technology, Instituto de Tecnologia Química e Biológica, Oeiras, Portugal)

## Project background
All investigated enzyme reaction without inhibitor showed progress curves behavior, which was not explainable by irreversible Michaelis-Menten kinetics. SUBSTRATE PRODUCT INHIBITION. All experiments had in commmon, that enzyme reactions were carried out in 96-well polystyrene micro titer plates (MTP), whereas the change in substrate and or product absorption was monitored photometrically. Initially, 
CotA laccase from *Bacillus subtilis*

__Show results of enzyme inactication across projects__ #TODO

All investigated enzyme reactions in this thesis showed a time-dependent decrease in catalytic activity, which was not explainable by the irreversible Michaelis-Menten model.

## Data Preparation

### Imports

In [12]:
import numpy as np
import pandas as pd
import pyenzyme as pe
from EnzymePynetics.tools.parameterestimator import ParameterEstimator
import matplotlib.pyplot as plt
from IPython.display import display


#TODO warning in EnzymePynetics

import warnings
warnings.filterwarnings('ignore')

### Experimantal data

Experimantal data was provided as an Excel file, containing time-course absorption data. Meta data was written into the EnzymeML Excel template. Then, the experimantal data was written to an EnzymeML document by a parser function. Concentrations were calculated via the provided extinction coefficient of the ABTS radical (ε = 36000 M<sup>-1</sup>cm<sup>-1</sup>). 

In [6]:
# Load experimental data from excel file
df = pd.read_excel("../../data/enzyme_inactivation/Repetition CotA ABTS kinetics higher volumes 2nd time.xlsx", sheet_name="csv").set_index("Time(min)")
data = df.values.T.reshape(8,4,72)
time = df.index.values

# Calculate concentrations
extinction_coefficient = 36 # (1/mM * 1/cm)
optical_length = 0.65 # cm

def absorption_to_concentration(abso):
    return abso / (extinction_coefficient*optical_length)

concentration_data = absorption_to_concentration(data)

# Parser function
def data_to_EnzymeML(
    template_path: str,
    measurement_data: np.ndarray,
    species_id: str,
    data_unit: str,
    time_unit: str
    ) -> pe.EnzymeMLDocument:

    enzmldoc = pe.EnzymeMLDocument.fromTemplate(template_path)
    for IDs, concentration in zip(enzmldoc.measurement_dict.keys(), measurement_data):
        for counter, replicate in enumerate(concentration):
            rep = pe.Replicate(
                id=f"Measurement{counter}",
                species_id=species_id,
                data=list(replicate),
                data_unit=data_unit,
                time=list(time),
                time_unit=time_unit)
            enzmldoc.getMeasurement(IDs).addReplicates(rep, enzmldoc)
    return enzmldoc

# Write experimantal data to EnzymeML document vis parser function
enzmldoc = data_to_EnzymeML(
    template_path="../../data/enzyme_inactivation/EnzymeML_CotA.xlsm",
    measurement_data=concentration_data,
    species_id="s1",
    data_unit="mmole / l",
    time_unit="min")

In [81]:
# Parameter estimation without time-dependent enzyme inactivation
CotA_kinetics = ParameterEstimator.from_EnzymeML(enzmldoc, "s1", "product")
CotA_kinetics.fit_models(enzyme_inactivation=False, display_output=False)
df = CotA_kinetics.result_dict.drop(columns=["kcat / Km [1/min * 1/mmole / l]"])
df.insert(1, "Enzyme inactivation", "False")

# Parameter estimation considering time-dependent enzyme inactivation
CotA_kinetics_with_inactivation = ParameterEstimator.from_EnzymeML(enzmldoc, "s1", "product")
CotA_kinetics_with_inactivation.fit_models(enzyme_inactivation=True, display_output=False)
df_inactivation = CotA_kinetics_with_inactivation.result_dict.drop(columns=["kcat / Km [1/min * 1/mmole / l]"])
df_inactivation.insert(1, "Enzyme inactivation", "True")

results = df.append(df_inactivation).sort_values("AIC")
display(results.style.set_table_attributes('style="font-size: 12px"'))

,AIC,Enzyme inactivation,kcat [1/min],Km [mmole / l],Ki competitive [mmole / l],Ki uncompetitive [mmole / l],ki time-dep enzyme-inactiv. [1/min]
substrate inhibition,-29337,True,3.209 +/- 0.90%,0.047 +/- 1.73%,-,33.750 +/- 13.18%,0.007 +/- 4.12%
competitive product inhibition,-29300,True,3.058 +/- 0.80%,0.037 +/- 3.55%,0.074 +/- 26.83%,-,0.006 +/- 5.27%
non-competitive product inhibition,-29296,True,3.077 +/- 2.06%,0.037 +/- 5.14%,0.064 +/- 40.49%,0.984 +/- 297.88%,0.006 +/- 29.07%
irreversible Michaelis Menten,-29287,True,3.095 +/- 0.77%,0.043 +/- 1.30%,-,-,0.007 +/- 4.18%
uncompetitive product inhibition,-29283,True,3.102 +/- 0.99%,0.043 +/- 2.20%,-,3.259 +/- 198.94%,0.007 +/- 5.60%
non-competitive product inhibition,-29276,False,3.312 +/- 1.52%,0.040 +/- 4.92%,0.024 +/- 12.73%,0.074 +/- 7.44%,nan
uncompetitive product inhibition,-29157,False,3.413 +/- 1.61%,0.062 +/- 2.46%,-,0.060 +/- 6.63%,nan
competitive product inhibition,-28969,False,2.638 +/- 0.33%,0.023 +/- 5.39%,0.013 +/- 11.76%,-,nan
substrate inhibition,-28778,False,2.664 +/- 0.59%,0.046 +/- 1.96%,-,37.058 +/- 16.47%,nan
irreversible Michaelis Menten,-28741,False,2.579 +/- 0.29%,0.042 +/- 1.48%,-,-,nan


,AIC,Enzyme inactivation,kcat [1/min],Km [mmole / l],kcat / Km [1/min * 1/mmole / l],Ki competitive [mmole / l],Ki uncompetitive [mmole / l],ki time-dep enzyme-inactiv. [1/min]
substrate inhibition,-29337,True,3.209 +/- 0.90%,0.047 +/- 1.73%,68.552 +/- 1.95%,-,33.750 +/- 13.18%,0.007 +/- 4.12%
competitive product inhibition,-29300,True,3.058 +/- 0.80%,0.037 +/- 3.55%,82.390 +/- 3.64%,0.074 +/- 26.83%,-,0.006 +/- 5.27%
non-competitive product inhibition,-29296,True,3.077 +/- 2.06%,0.037 +/- 5.14%,82.166 +/- 5.54%,0.064 +/- 40.49%,0.984 +/- 297.88%,0.006 +/- 29.07%
irreversible Michaelis Menten,-29287,True,3.095 +/- 0.77%,0.043 +/- 1.30%,72.588 +/- 1.51%,-,-,0.007 +/- 4.18%
uncompetitive product inhibition,-29283,True,3.102 +/- 0.99%,0.043 +/- 2.20%,72.114 +/- 2.41%,-,3.259 +/- 198.94%,0.007 +/- 5.60%
non-competitive product inhibition,-29276,False,3.312 +/- 1.52%,0.040 +/- 4.92%,82.445 +/- 5.15%,0.024 +/- 12.73%,0.074 +/- 7.44%,NaN
uncompetitive product inhibition,-29157,False,3.413 +/- 1.61%,0.062 +/- 2.46%,54.871 +/- 2.94%,-,0.060 +/- 6.63%,NaN
competitive product inhibition,-28969,False,2.638 +/- 0.33%,0.023 +/- 5.39%,112.573 +/- 5.40%,0.013 +/- 11.76%,-,NaN
substrate inhibition,-28778,False,2.664 +/- 0.59%,0.046 +/- 1.96%,57.611 +/- 2.05%,-,37.058 +/- 16.47%,NaN
irreversible Michaelis Menten,-28741,False,2.579 +/- 0.29%,0.042 +/- 1.48%,60.734 +/- 1.50%,-,-,NaN


- Paulo
    - Long measurement over 70 min. 
    - enzyme inacitivation model better fit with less parameters.
    - 

In [8]:
def get_sub(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans(''.join(normal), ''.join(sub_s))
    return x.translate(res)
print(get_sub("cat"))




꜀ₐₜ
